## Setup

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop
from kepler.pandas.decorators import RingerDecorator
from itertools import product
import rootplotlib as rpl
import mplhep as hep
import root_numpy
import ROOT
ROOT.gStyle.SetOptStat(0);
import numpy as np
import pandas as pd
import glob
import os
import logging
import matplotlib.pyplot as plt
plt.style.use(hep.style.ROOT)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict

from packages.generators import ring_percentages, RingGenerator
from packages.plotting import make_plot_fig, cached_root2fig
from packages.utils import get_logger

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


In [2]:
drop_cols = drop_columns = [
                    'RunNumber', 
                    'trig_L2_cl_e2tsts1',
                    'trig_L2_el_hastrack',
                    'trig_L2_el_eta',
                    'trig_L2_el_phi',
                    'trig_L2_el_caloEta',
                    'trig_L2_el_trkClusDeta',
                    'trig_L2_el_trkClusDphi',
                    'trig_L2_el_etOverPt',
                    'trig_EF_cl_hascluster',
                    'trig_EF_cl_eta',
                    'trig_EF_cl_etaBE2',
                    'trig_EF_cl_phi',     
                    'trig_EF_el_hascand',
                    'trig_EF_el_eta',
                    'trig_EF_el_etaBE2',
                    'trig_EF_el_phi',
                    'trig_EF_el_rhad1',
                    'trig_EF_el_rhad',
                    'trig_EF_el_f3',
                    'trig_EF_el_weta2',
                    'trig_EF_el_rphi',
                    'trig_EF_el_reta',
                    'trig_EF_el_wtots1',
                    'trig_EF_el_eratio',
                    'trig_EF_el_f1',
                    'trig_EF_el_hastrack',
                    'trig_EF_el_deltaEta1',
                    'trig_EF_el_deltaPhi2',
                    'trig_EF_el_deltaPhi2Rescaled',
                    'el_etaBE2',
                    'el_numberOfBLayerHits',
                    'el_numberOfPixelHits',
                    'el_numberOfTRTHits',
                    'el_trans_TRT_PID',
                    'el_deltaPhi2',
                    'el_TaP_Mass',
                ]

l1seeds_per_energy = {
    24: 'L1_EM22VHI',
    26: 'L1_EM22VHI',
    60: 'L1_EM24VHI',
    140: 'L1_EM24VHI'
}

criteria_conf_names = {
    'tight': 'ElectronRingerTightTriggerConfig.conf',
    'medium': 'ElectronRingerMediumTriggerConfig.conf',
    'loose': 'ElectronRingerLooseTriggerConfig.conf',
    'vloose': 'ElectronRingerVeryLooseTriggerConfig.conf'
}

energy_chains = {
     24: 'e24_lhtight_nod0_{strategy}_ivarloose',
     26: 'e26_lhtight_nod0_{strategy}_ivarloose',
     60: 'e60_lhmedium_nod0_{strategy}_L1EM24VHI',
     140: 'e140_lhloose_nod0_{strategy}'
}

steps_choices = ['L2Calo', 'L2', 'EFCalo', 'HLT']

## Parameters

In [3]:
HOME_PATH = os.path.expanduser('~')
datasetpath = os.path.join(HOME_PATH, 
    '/data/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins.v2')

modelpaths = [
    'tunings/TrigL2_20180125_v8',
    'tunings/TrigL2_20210907_v12 (v8.1-2)',
    'tunings/TrigL2_20220704_v20',
    'tunings/TrigL2_20220802_v20.1'
    # 'tunings/TrigL2_20210306_v9'
]

output_dir = 'analysis/chain_simulation_test'
cutbased = True
plot_vars = ['et', 'eta', 'pt', 'mu']
values = ['pd', 'fr']

chain_names = [
    'e24_lhtight_nod0_{strategy}_ivarloose',
    'e26_lhtight_nod0_{strategy}_ivarloose',
    'e60_lhmedium_nod0_{strategy}_L1EM24VHI',
    'e140_lhloose_nod0_{strategy}'
]

trigger_steps = ['L2Calo', 'L2', 'EFCalo', 'HLT']
dev = True
log = False

# Initializing logger
analysis_logger = get_logger('run_analysis', file=log)


## Building Decorators

In [7]:
analysis_logger.info('Building decorators')
decorators = list()
trigger_strategies = ['noringer'] if cutbased else list()
for modelpath, criterion in product(modelpaths, criteria_conf_names.keys()):
    conf_name = criteria_conf_names[criterion]
    confpath = os.path.join(modelpath, conf_name)
    env = ROOT.TEnv(confpath)
    ringer_version = env.GetValue("__version__", '')
    ringer_name = f'ringer_{ringer_version}'
    start_criterion = f'{ringer_name}_{criterion}'
    analysis_logger.info(f'Building decorator for {confpath}. Version: {ringer_version}')
    decorator = RingerDecorator(start_criterion, confpath, RingGenerator(ring_percentages[ringer_version]))
    decorators.append(decorator)
    if not ringer_name in trigger_strategies:
        trigger_strategies.append(ringer_name)

2022-08-29 13:55:48,364 - Building decorators
2022-08-29 13:55:48,415 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerTightTriggerConfig.conf. Version: v8
2022-08-29 13:55:48,778 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerMediumTriggerConfig.conf. Version: v8
2022-08-29 13:55:49,130 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerLooseTriggerConfig.conf. Version: v8
2022-08-29 13:55:49,483 - Building decorator for tunings/TrigL2_20180125_v8/ElectronRingerVeryLooseTriggerConfig.conf. Version: v8
2022-08-29 13:55:49,835 - Building decorator for tunings/TrigL2_20210907_v12 (v8.1-2)/ElectronRingerTightTriggerConfig.conf. Version: v12
2022-08-29 13:55:50,176 - Building decorator for tunings/TrigL2_20210907_v12 (v8.1-2)/ElectronRingerMediumTriggerConfig.conf. Version: v12
2022-08-29 13:55:50,514 - Building decorator for tunings/TrigL2_20210907_v12 (v8.1-2)/ElectronRingerLooseTriggerConfig.conf. Version: v12
2022-08-29 13:55:50,854 - 

2022-08-29 13:55:48,417 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerTightTriggerConfig.conf
2022-08-29 13:55:48,779 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerMediumTriggerConfig.conf
2022-08-29 13:55:49,131 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerLooseTriggerConfig.conf
2022-08-29 13:55:49,485 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20180125_v8/ElectronRingerVeryLooseTriggerConfig.conf
2022-08-29 13:55:49,836 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20210907_v12 (v8.1-2)/ElectronRingerTightTriggerConfig.conf
2022-08-29 13:55:50,177 | Py.RingerDecorator                      INFO Reading... tunings/TrigL2_20210907_v12 (v8.1-2)/ElectronRingerMediumTriggerConfig.conf
2022-08-29 13:55:50,516 | Py.RingerDecorator                      INFO Reading... tunings/Tr

## Building Chains

In [10]:
analysis_logger.info('Building chains')
chains = list()
step_chain_names = list()
for chain_name, strategy in product(chain_names, trigger_strategies):
    spliited_chain_name = chain_name.split('_')
    criterion = spliited_chain_name[1].replace('lh', '')
    step_chain_name = f'HLT_{chain_name.format(strategy=strategy)}'
    step_chain_names.append(step_chain_name)
    energy = int(spliited_chain_name[0][1:])
    l1seed = l1seeds_per_energy[energy]
    l2calo_column = f'{strategy}_{criterion}'
    analysis_logger.info(f'Building chain: {step_chain_name} model: {l2calo_column}')
    if strategy == 'noringer':
        chain = Chain(step_chain_name, L1Seed=l1seed)
    else:
        chain = Chain(step_chain_name, L1Seed=l1seed, l2calo_column=l2calo_column)
    chains.append(chain)

2022-08-29 13:57:15,265 - Building chains
2022-08-29 13:57:15,268 - Building chain: HLT_e24_lhtight_nod0_noringer_ivarloose model: noringer_tight
2022-08-29 13:57:15,272 - Building chain: HLT_e24_lhtight_nod0_ringer_v8_ivarloose model: ringer_v8_tight
2022-08-29 13:57:15,275 - Building chain: HLT_e24_lhtight_nod0_ringer_v12_ivarloose model: ringer_v12_tight
2022-08-29 13:57:15,278 - Building chain: HLT_e24_lhtight_nod0_ringer_v20_ivarloose model: ringer_v20_tight
2022-08-29 13:57:15,279 - Building chain: HLT_e24_lhtight_nod0_ringer_v20.1_ivarloose model: ringer_v20.1_tight
2022-08-29 13:57:15,284 - Building chain: HLT_e26_lhtight_nod0_noringer_ivarloose model: noringer_tight
2022-08-29 13:57:15,285 - Building chain: HLT_e26_lhtight_nod0_ringer_v8_ivarloose model: ringer_v8_tight
2022-08-29 13:57:15,286 - Building chain: HLT_e26_lhtight_nod0_ringer_v12_ivarloose model: ringer_v12_tight
2022-08-29 13:57:15,288 - Building chain: HLT_e26_lhtight_nod0_ringer_v20_ivarloose model: ringer_v20_

OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'trig_L2_cl_lhtight_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v8_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v12_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v20_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1

## Reading Data and Simulating Chains

In [12]:
analysis_logger.info('Loading the data')
filename_end = '*et4_eta4.npz' if dev else '*.npz'    #If dev, loads leblon
datafiles = glob.glob(os.path.join(datasetpath, filename_end))  
analysis_logger.info(f'glob_path: {os.path.join(datasetpath, filename_end)}')
data = load_in_loop(datafiles, drop_columns=drop_cols, decorators=decorators, chains=chains)

2022-08-29 14:01:39,763 - Loading the data
2022-08-29 14:01:39,767 - glob_path: /home/lbarranunes/data/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins.v2/*et4_eta4.npz
Reading files...:   0%|          | 0/1 [00:00<?, ?it/s]

L1_EM22VHI trig_L2_cl_lhtight_et22toInf trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v12_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v20_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v20.1_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI trig_L2_cl_lhtight_et22toInf trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v12_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v20_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v20.1_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM24VHI trig_L2_cl_lhmedium_et22toInf trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v8_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM

Reading files...: 100%|██████████| 1/1 [00:02<00:00,  2.04s/it]

L1_EM24VHI ringer_v20_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose
L1_EM24VHI ringer_v20.1_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose


## Plotting

In [7]:
analysis_logger.info('Making plots')
for value, var, chain_name, step in product(values, plot_vars, chain_names, trigger_steps):
    plot_dir = os.path.join(output_dir, value, var)
    analysis_logger.info(f'Plotting value: {value}, step: {step}, chain_name: {chain_name}, var: {var}')
    make_plot_fig(data, step, chain_name, trigger_strategies, plot_dir , var, value, joblib_dump=True)

2022-08-17 21:41:19,544 - Making plots
2022-08-17 21:41:19,546 - Plotting value: pd, step: L2Calo, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:20,211 - Plotting value: pd, step: L2, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:20,618 - Plotting value: pd, step: EFCalo, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:20,999 - Plotting value: pd, step: HLT, chain_name: e24_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:21,357 - Plotting value: pd, step: L2Calo, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:21,756 - Plotting value: pd, step: L2, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:22,146 - Plotting value: pd, step: EFCalo, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:22,527 - Plotting value: pd, step: HLT, chain_name: e26_lhtight_nod0_{strategy}_ivarloose, var: et
2022-08-17 21:41:22,888

## Regenerate from cached data

In [3]:
analysis_logger = get_logger('cache_analysis', file=False)
colors = [1, 601, 616, 401]
markers = [33, 22, 20, 34]

In [4]:
analysis_directory = os.path.join('analysis', '2022_08_18_v20.1_data17')
for plot_label in glob.iglob(os.path.join(analysis_directory, '**', '**', '*_labels.joblib')):
    plot_dir, plot_name = os.path.split(plot_label)
    plot_name = plot_name.replace('_labels.joblib', '')
    analysis_logger.info(f'Processing {plot_name}')
    cached_root2fig(plot_dir, plot_name, colors, markers, save=True)

2022-08-23 19:28:07,592 - Processing et_pd_L2Calo_e24_lhtight
2022-08-23 19:28:07,792 - Processing et_pd_L2_e24_lhtight
2022-08-23 19:28:07,850 - Processing et_pd_EFCalo_e24_lhtight
2022-08-23 19:28:07,912 - Processing et_pd_HLT_e24_lhtight
2022-08-23 19:28:07,997 - Processing et_pd_L2Calo_e26_lhtight
2022-08-23 19:28:08,031 - Processing et_pd_L2_e26_lhtight
2022-08-23 19:28:08,100 - Processing et_pd_EFCalo_e26_lhtight
2022-08-23 19:28:08,179 - Processing et_pd_HLT_e26_lhtight
2022-08-23 19:28:08,214 - Processing et_pd_L2Calo_e60_lhmedium
2022-08-23 19:28:08,300 - Processing et_pd_L2_e60_lhmedium
2022-08-23 19:28:08,331 - Processing et_pd_EFCalo_e60_lhmedium
2022-08-23 19:28:08,367 - Processing et_pd_HLT_e60_lhmedium
2022-08-23 19:28:08,399 - Processing et_pd_L2Calo_e140_lhloose
2022-08-23 19:28:08,432 - Processing et_pd_L2_e140_lhloose
2022-08-23 19:28:08,464 - Processing et_pd_EFCalo_e140_lhloose
2022-08-23 19:28:08,508 - Processing et_pd_HLT_e140_lhloose
2022-08-23 19:28:08,560 - Pr